<a href="https://colab.research.google.com/github/AlexanderZobkov/ikea-stock-reporter/blob/master/ikea_stock_reporter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd

In [2]:
items = {
    '10014030': "Каркас гардероба, белый, 50x58x236 см",
    '40277973': "Полка, белый, 50x58 см",
    '20382343': "4 панели д/рамы раздвижной дверцы, глянцевый светло-серый, 100x236 см"
}
stores = {
    '335': 'Химки',
    '336': 'Теплый Стан',
    '344': 'Белая Дача',
}


In [3]:
items_url = ','.join(list(items.keys()))
availability_url = f'https://api.ingka.ikea.com/cia/availabilities/ru/ru?itemNos={items_url}&expand=StoresList,Restocks'

In [4]:
ingka_headers = {
    'Connection': 'close',
    'Accept': 'application/json;version=2',
    'Referer': 'https://www.ikea.com/',
    'X-Client-Id': 'b6c117e5-ae61-4ef5-b4cc-e0b1e37f0631',
}

In [6]:
try:
    response = requests.get(availability_url, headers=ingka_headers)
    response.raise_for_status()
    ingka_json_response = response.json()
    availability_in_stores_of_interest = list(
        filter(lambda availability: (availability['classUnitKey']['classUnitType'] == 'STO' and
                                     availability['classUnitKey']['classUnitCode'] in list(stores.keys()) and
                                     availability['itemKey']['itemType'] in ['ART', 'SPR']),
               ingka_json_response['availabilities']))

    products = []
    product_ids = []
    store = []
    stock = []
    for availability in availability_in_stores_of_interest:
        products.append(items[availability['itemKey']['itemNo']])
        product_ids.append(availability['itemKey']['itemNo'])
        store.append(stores[availability['classUnitKey']['classUnitCode']])
        stock.append(availability['buyingOption']['cashCarry']['availability']['quantity'])

    df = pd.DataFrame({'Product': products, 'Product Id': product_ids, 'Store': store, 'Stock': stock})
    table = pd.pivot_table(
        df,
        values=["Stock"],
        index=["Product", "Product Id"],
        columns=["Store"],
    )
    print(table)

except requests.exceptions.HTTPError as error:
    print(error)

                                                      Stock                  
Store                                            Белая Дача Теплый Стан Химки
Product                               Product Id                             
Каркас гардероба, белый, 50x58x236 см 10014030            0          24    74
Полка, белый, 50x58 см                40277973         2533        2393     0
